In [1]:
# Load packages:
import numpy as np
import pandas as pd
from datetime import date
from statistics import mode, StatisticsError
from collections import Counter
import datetime

## Table of content:

- Load datasets
- Aggregating codes according to a priori/technical criterion --> PDM
- Resampling log data into daily series
- Resampling log data into weekly series
- New variables creation

In order to perform the analysis, it is necessary to:
- create additional variables to catch eterogeneity of data
- aggregate data to overcome to the irregularity of the time series --> undersampling
- aggregate data for the parsimony of the model

### Load dataset

In [2]:
# Load all datasets:
df = pd.read_parquet('Final_dataset//TSR_040_DEF.parquet', engine='fastparquet')
corr_maint = pd.read_parquet('Final_dataset//correct_maint_040_TCU.parquet', engine='fastparquet')
sched_maint = pd.read_parquet('Final_dataset//sched_maint_040.parquet', engine='fastparquet')
routes = pd.read_parquet('Final_dataset//routes_040_uptofermo.parquet', engine='fastparquet')
code = pd.read_csv('..//Codici_diagnostici//Codici_Diagnostici.csv', delimiter=",")

In [3]:
# check:
print(df.info())
print(corr_maint.info())
print(sched_maint.info())
print(routes.info())
print(code.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101708 entries, 0 to 101707
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Source         101708 non-null  category      
 1   Name           101708 non-null  category      
 2   Time           101708 non-null  datetime64[ns]
 3   Event_type     101708 non-null  category      
 4   Type           101708 non-null  category      
 5   Real_Monitor   101708 non-null  category      
 6   cod            101708 non-null  int64         
 7   id             101708 non-null  int64         
 8   id1            101708 non-null  int64         
 9   Complete_code  101708 non-null  category      
 10  Master         101708 non-null  category      
 11  Speed          101708 non-null  int64         
 12  Depot          101708 non-null  category      
 13  Lat            101708 non-null  float64       
 14  Lon            101708 non-null  float64       
 15  

In [72]:
'''
# Load all datasets:
df = pd.read_csv('Final_dataset//TSR_040_DEF.csv', delimiter=',', dtype={
    'Source': 'category',
    'Name': 'category',
    'Event_type': 'category',
    'Type': 'category',
    'Real_Monitor': 'category',
    'Complete_code': 'category',
    'Master': 'category',
    'Depot': 'category',
    'Criticality': 'category',
    'Colour': 'category'
})
df["Time"] = pd.to_datetime(df["Time"], format='%Y-%m-%d %H:%M:%S')

corr_maint = pd.read_csv('Final_dataset//correct_maint_040_TCU.csv', delimiter=',', dtype={
    'Train': 'category',
    'Wagon': 'category',
    'Type': 'category',
    'Group_maint_source': 'category',
})
corr_maint["End_date"] = pd.to_datetime(corr_maint["End_date"], format='%Y-%m-%d')
corr_maint["Start_date"] = pd.to_datetime(corr_maint["Start_date"], format='%Y-%m-%d')
corr_maint["Duration"] = corr_maint["Duration"].astype('timedelta64[s]')

sched_maint = pd.read_csv('Final_dataset//sched_maint_040.csv', delimiter=',', dtype={
    'Train': 'category',
    'Type': 'category',
    'Level': 'category',
})
sched_maint["End_date"] = pd.to_datetime(sched_maint["End_date"], format='%Y-%m-%d')
sched_maint["Start_date"] = pd.to_datetime(sched_maint["Start_date"], format='%Y-%m-%d')
sched_maint["Duration"] = sched_maint["Duration"].astype('timedelta64[s]')

routes = pd.read_csv('Final_dataset//routes_040_uptofermo.csv', delimiter=',')
routes["Date"] = pd.to_datetime(routes["Date"], format='%Y-%m-%d')
'''

'\n# Load all datasets:\ndf = pd.read_csv(\'Final_dataset//TSR_040_DEF.csv\', delimiter=\',\', dtype={\n    \'Source\': \'category\',\n    \'Name\': \'category\',\n    \'Event_type\': \'category\',\n    \'Type\': \'category\',\n    \'Real_Monitor\': \'category\',\n    \'Complete_code\': \'category\',\n    \'Master\': \'category\',\n    \'Depot\': \'category\',\n    \'Criticality\': \'category\',\n    \'Colour\': \'category\'\n})\ndf["Time"] = pd.to_datetime(df["Time"], format=\'%Y-%m-%d %H:%M:%S\')\n\ncorr_maint = pd.read_csv(\'Final_dataset//correct_maint_040_TCU.csv\', delimiter=\',\', dtype={\n    \'Train\': \'category\',\n    \'Wagon\': \'category\',\n    \'Type\': \'category\',\n    \'Group_maint_source\': \'category\',\n})\ncorr_maint["End_date"] = pd.to_datetime(corr_maint["End_date"], format=\'%Y-%m-%d\')\ncorr_maint["Start_date"] = pd.to_datetime(corr_maint["Start_date"], format=\'%Y-%m-%d\')\ncorr_maint["Duration"] = corr_maint["Duration"].astype(\'timedelta64[s]\')\n\nsched_

## Aggregation into PDM:

In order to perform the analysis and apply machine learning algorithm, an aggregation based on a priori criterion is performed. Which is aggregate from PDO into PDM but considering the following:

- warnings --> warnings should be still separated from alerts:
    - for the 5-12-1 there is no problem because this pdo is the only one under the PDM 5-12
    - while there is a little issue with 5-98-11 and 5-98-12 because they are 2 warnings among several PDO under the PDM 5-18 --> in order to correctly label them as warning, 2 fake PDM 5-20 and 5-21 are created
- train_driver --> train_driver should be still separated from alerts and warnings: no problem because 5-17 is originally a PDM
- fake PDM 5-31,....,5-99: we assume they are truly PDM because otherwise many alerts would be lost
- "Blocco temporaneo" and "Blocco permanente" alerts: we assume that each PDO generates the above PDM

In [73]:
# 1) check:
print(df.info())

print("\n")
print("-complete_code:")
print(f"n. of unique values: {df['Complete_code'].nunique()}")

print("\n")
print("-cod:")
print(f"n. of unique values: {df['cod'].nunique()}")
print(df['cod'].value_counts())

print("\n")
print("-id:")
print(f"n. of unique values: {df['id'].nunique()}")
print(df['id'].value_counts())

print("\n")
print("-id1:")
print(f"n. of unique values: {df['id1'].nunique()}")
print(df['id1'].value_counts())

print("\n")
print("-more detailed:")
print("cod=2:")
print(f"how many complete_code: {df[df['cod']==2]['Complete_code'].nunique()}")
print(f"how many id: {df[df['cod']==2]['id'].nunique()}")
print(f"how many id1: {df[df['cod']==2]['id1'].nunique()}")
print("cod=5:")
print(f"how many complete_code: {df[df['cod']==5]['Complete_code'].nunique()}")
print(f"how many id: {df[df['cod']==5]['id'].nunique()}")
print(f"how many id1: {df[df['cod']==5]['id1'].nunique()}")

print("\n")
print("-recap:")
print(f"how many pdm should result: {df['id1'].value_counts().sum()}, which is the len of the dataset!")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101708 entries, 0 to 101707
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Source         101708 non-null  category      
 1   Name           101708 non-null  category      
 2   Time           101708 non-null  datetime64[ns]
 3   Event_type     101708 non-null  category      
 4   Type           101708 non-null  category      
 5   Real_Monitor   101708 non-null  category      
 6   cod            101708 non-null  int64         
 7   id             101708 non-null  int64         
 8   id1            101708 non-null  int64         
 9   Complete_code  101708 non-null  category      
 10  Master         101708 non-null  category      
 11  Speed          101708 non-null  int64         
 12  Depot          101708 non-null  category      
 13  Lat            101708 non-null  float64       
 14  Lon            101708 non-null  float64       
 15  

In [74]:
# 2) drop columns Complete_code, colour and criticality because they will be again created as last step:
df2 = df.drop(columns=(['Complete_code', 'Colour', 'Criticality']))
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101708 entries, 0 to 101707
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Source        101708 non-null  category      
 1   Name          101708 non-null  category      
 2   Time          101708 non-null  datetime64[ns]
 3   Event_type    101708 non-null  category      
 4   Type          101708 non-null  category      
 5   Real_Monitor  101708 non-null  category      
 6   cod           101708 non-null  int64         
 7   id            101708 non-null  int64         
 8   id1           101708 non-null  int64         
 9   Master        101708 non-null  category      
 10  Speed         101708 non-null  int64         
 11  Depot         101708 non-null  category      
 12  Lat           101708 non-null  float64       
 13  Lon           101708 non-null  float64       
 14  Lat_umt       101708 non-null  float64       
 15  Lon_umt       101

In [75]:
# 3) create 2 fake pdm 5-20 and 5-21 in order to separate the 2 warnings 5-98-11 and 5-98-12 from the pdm 5-98:
df2['id'] = df2.apply(lambda row: 20 if row['id'] == 98 and row['id1'] == 11 else row['id'], axis=1)
df2['id'] = df2.apply(lambda row: 21 if row['id'] == 98 and row['id1'] == 12 else row['id'], axis=1)
df2['id1'] = df2.apply(lambda row: 0 if row['id'] in (20, 21) else row['id1'], axis=1)

# check:
print(len(df[(df['id']==98) & (df['id1']==11)]))
print(len(df2[(df2['id']==20) & (df2['id1']==0)]))
print(len(df[(df['id']==98) & (df['id1']==12)]))
print(len(df2[(df2['id']==21) & (df2['id1']==0)]))

714
714
644
644


In [76]:
# 3) assign value 0 to column id1 for each row:
df2['id1'] = df2.apply(lambda row: 0, axis=1)

# check:
print(df2['id1'].nunique())
print(df2['id1'].value_counts())

1
id1
0    101708
Name: count, dtype: int64


In [77]:
# 4) create variable "Complete_code":
df2['Complete_code'] = df2.apply(lambda row: f"{row['cod']}-{row['id']}" if row['id1'] == 0 else f"{row['cod']}-{row['id']}-{row['id1']}", axis=1)
df2['Complete_code'] = df2['Complete_code'].astype('category')

# check:
df2

,Source,Name,Time,Event_type,Type,Real_Monitor,cod,id,id1,Master,Speed,Depot,Lat,Lon,Lat_umt,Lon_umt,Complete_code
0,711-083,711-083,2020-11-12 06:54:18,ON,PDO,MD,5,5,0,711-083,49,0,45.4395,9.23639,518488.162378,5.031802e+06,5-5
1,711-083,711-083,2020-11-12 06:54:18,ON,PDO,MD,5,98,0,711-083,49,0,45.4395,9.23639,518488.162378,5.031802e+06,5-98
2,711-084,710-132,2020-11-12 10:50:09,ON,PDO,MD,5,98,0,711-084,0,0,45.5789,9.15668,512223.798419,5.047274e+06,5-98
3,711-084,710-170,2020-11-12 10:57:17,ON,PDO,MD,5,11,0,711-084,61,0,45.5578,9.16639,512986.210566,5.044931e+06,5-11
4,711-084,710-170,2020-11-12 10:57:17,ON,PDO,MD,5,11,0,711-084,61,0,45.5578,9.16639,512986.210566,5.044931e+06,5-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101703,711-084,710-170,2023-09-27 12:55:35,ON,PDO,MD,5,99,0,711-084,15,0,45.4711,9.17230,513468.132303,5.035300e+06,5-99
101704,711-084,710-132,2023-09-27 12:55:35,ON,PDO,MD,5,99,0,711-084,15,0,45.4711,9.17230,513468.132303,5.035300e+06,5-99
101705,711-084,711-084,2023-09-27 12:55:35,ON,PDO,MD,5,99,0,711-084,15,0,45.4711,9.17230,513468.132303,5.035300e+06,5-99
101706,711-084,711-084,2023-09-27 19:45:41,ON,PDO,MD,5,98,0,711-084,0,0,45.4711,9.17231,513468.913971,5.035300e+06,5-98


In [78]:
# 5) change value "PDO" into "PDM" in Type variable:
df2['Type'] = df2.apply(lambda row: 'PDM' if row['Type']=='PDO' else row['Type'], axis=1)

# check:
print(df['Type'].nunique())
print(df['Type'].value_counts())
print(df2['Type'].nunique())
print(df2['Type'].value_counts())

3
Type
PDO             99739
WARNING          1548
TRAIN_DRIVER      421
Name: count, dtype: int64
3
Type
PDM             99739
WARNING          1548
TRAIN_DRIVER      421
Name: count, dtype: int64


In [79]:
# 6) merge:
df3 = pd.merge(df2, code, left_on='Complete_code', right_on='Code')
df3 = df3.drop(columns=['Code'])
df3

,Source,Name,Time,Event_type,Type,Real_Monitor,cod,id,id1,Master,Speed,Depot,Lat,Lon,Lat_umt,Lon_umt,Complete_code,Criticality,Colour
0,711-083,711-083,2020-11-12 06:54:18,ON,PDM,MD,5,5,0,711-083,49,0,45.4395,9.23639,518488.162378,5.031802e+06,5-5,1,giallo
1,711-083,711-083,2020-11-12 06:54:18,ON,PDM,MD,5,98,0,711-083,49,0,45.4395,9.23639,518488.162378,5.031802e+06,5-98,0,azzurro
2,711-084,710-132,2020-11-12 10:50:09,ON,PDM,MD,5,98,0,711-084,0,0,45.5789,9.15668,512223.798419,5.047274e+06,5-98,0,azzurro
3,711-084,710-170,2020-11-12 10:57:17,ON,PDM,MD,5,11,0,711-084,61,0,45.5578,9.16639,512986.210566,5.044931e+06,5-11,0,giallo
4,711-084,710-170,2020-11-12 10:57:17,ON,PDM,MD,5,11,0,711-084,61,0,45.5578,9.16639,512986.210566,5.044931e+06,5-11,0,giallo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101703,711-084,710-170,2023-09-27 12:55:35,ON,PDM,MD,5,99,0,711-084,15,0,45.4711,9.17230,513468.132303,5.035300e+06,5-99,0,azzurro
101704,711-084,710-132,2023-09-27 12:55:35,ON,PDM,MD,5,99,0,711-084,15,0,45.4711,9.17230,513468.132303,5.035300e+06,5-99,0,azzurro
101705,711-084,711-084,2023-09-27 12:55:35,ON,PDM,MD,5,99,0,711-084,15,0,45.4711,9.17230,513468.132303,5.035300e+06,5-99,0,azzurro
101706,711-084,711-084,2023-09-27 19:45:41,ON,PDM,MD,5,98,0,711-084,0,0,45.4711,9.17231,513468.913971,5.035300e+06,5-98,0,azzurro


In [80]:
# 7) final check:
print(len(df3))
print("\n")
print(df3['Complete_code'].nunique())
print("\n")
print(df3[df3['cod']==2]['Complete_code'].nunique())
print(df3[df3['cod']==5]['Complete_code'].nunique())
print("\n")
print(df3['Complete_code'].value_counts())
print("\n")
print(len(df[df['id']==98]))
print(len(df[(df['id']==98) & (df['id1']==11)]))
print(len(df[(df['id']==98) & (df['id1']==12)]))
print(len(df3[df3['id']==98]))

print(df3.info())
df3['Type'] = df3['Type'].astype('category')
df3['Complete_code'] = df3['Complete_code'].astype('category')
df3['Criticality'] = df3['Criticality'].astype('category')
df3['Colour'] = df3['Colour'].astype('category')
print(df3.info())

101708


25


1
24


Complete_code
5-98    26266
5-5     11735
5-3     10439
5-9     10126
5-35     7438
5-33     6488
5-99     4028
5-13     3931
5-7      3849
5-31     3007
5-1      2874
5-11     1762
5-34     1430
2-19     1380
5-32     1320
5-14      948
5-15      907
5-36      739
5-20      714
5-21      644
5-10      496
5-17      421
5-16      397
5-12      190
5-90      179
Name: count, dtype: int64


27624
714
644
26266
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101708 entries, 0 to 101707
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Source         101708 non-null  category      
 1   Name           101708 non-null  category      
 2   Time           101708 non-null  datetime64[ns]
 3   Event_type     101708 non-null  category      
 4   Type           101708 non-null  object        
 5   Real_Monitor   101708 non-null  category      
 6   cod            101708 non-null  int

### Resampling by DAY --> daily observations
### Resampling by WEEK --> weekly observations

#### PDM aggregated dataset:

In [81]:
# 1) function to compute the mode:
def calculate_mode(series):
    try:
        return mode(series)
    except StatisticsError:
        return series.mode().iloc[0] if not series.mode().empty else None

In [82]:
# 2) function to simplify the typing procedure:
def labels_cat_var(df, column_name):
    result = ""
    for i, cat in enumerate(df[column_name].cat.categories):
        result += f"('{column_name}_{cat}', lambda x: (x == df3['{column_name}'].cat.categories[{i}]).sum()),"
    return result

# and apply it:
for col in df3.columns:
    if isinstance(df3[col].dtype, pd.CategoricalDtype):
        print(labels_cat_var(df3,col), "\n", "\n")

('Source_711-083', lambda x: (x == df3['Source'].cat.categories[0]).sum()),('Source_711-084', lambda x: (x == df3['Source'].cat.categories[1]).sum()), 
 

('Name_710-131', lambda x: (x == df3['Name'].cat.categories[0]).sum()),('Name_710-132', lambda x: (x == df3['Name'].cat.categories[1]).sum()),('Name_710-135', lambda x: (x == df3['Name'].cat.categories[2]).sum()),('Name_710-170', lambda x: (x == df3['Name'].cat.categories[3]).sum()),('Name_711-083', lambda x: (x == df3['Name'].cat.categories[4]).sum()),('Name_711-084', lambda x: (x == df3['Name'].cat.categories[5]).sum()),('Name_710-000', lambda x: (x == df3['Name'].cat.categories[6]).sum()), 
 

('Event_type_OFF', lambda x: (x == df3['Event_type'].cat.categories[0]).sum()),('Event_type_ON', lambda x: (x == df3['Event_type'].cat.categories[1]).sum()), 
 

('Type_PDM', lambda x: (x == df3['Type'].cat.categories[0]).sum()),('Type_TRAIN_DRIVER', lambda x: (x == df3['Type'].cat.categories[1]).sum()),('Type_WARNING', lambda x: (x == df3['

In [83]:
# 3) definition of aggregation functions to use:
agg_funcs = {
    'Source': [
        ('Source_mode', calculate_mode),
        ('Source_711-083', lambda x: (x == df3['Source'].cat.categories[0]).sum()),('Source_711-084', lambda x: (x == df3['Source'].cat.categories[1]).sum()),
    ],
    
    'Name': [
        ('Name_mode', calculate_mode),
        ('Name_710-131', lambda x: (x == df3['Name'].cat.categories[0]).sum()),('Name_710-132', lambda x: (x == df3['Name'].cat.categories[1]).sum()),('Name_710-135', lambda x: (x == df3['Name'].cat.categories[2]).sum()),('Name_710-170', lambda x: (x == df3['Name'].cat.categories[3]).sum()),('Name_711-083', lambda x: (x == df3['Name'].cat.categories[4]).sum()),('Name_711-084', lambda x: (x == df3['Name'].cat.categories[5]).sum()),
    ],
    
    'Type': [
        ('Type_mode', calculate_mode),
        ('Type_PDM', lambda x: (x == df3['Type'].cat.categories[0]).sum()),('Type_TRAIN_DRIVER', lambda x: (x == df3['Type'].cat.categories[1]).sum()),('Type_WARNING', lambda x: (x == df3['Type'].cat.categories[2]).sum()),
    ],

    'Complete_code': [
        ('Complete_code_mode', calculate_mode),
        ('Complete_code_2-19', lambda x: (x == df3['Complete_code'].cat.categories[0]).sum()),('Complete_code_5-1', lambda x: (x == df3['Complete_code'].cat.categories[1]).sum()),('Complete_code_5-10', lambda x: (x == df3['Complete_code'].cat.categories[2]).sum()),('Complete_code_5-11', lambda x: (x == df3['Complete_code'].cat.categories[3]).sum()),('Complete_code_5-12', lambda x: (x == df3['Complete_code'].cat.categories[4]).sum()),('Complete_code_5-13', lambda x: (x == df3['Complete_code'].cat.categories[5]).sum()),('Complete_code_5-14', lambda x: (x == df3['Complete_code'].cat.categories[6]).sum()),('Complete_code_5-15', lambda x: (x == df3['Complete_code'].cat.categories[7]).sum()),('Complete_code_5-16', lambda x: (x == df3['Complete_code'].cat.categories[8]).sum()),('Complete_code_5-17', lambda x: (x == df3['Complete_code'].cat.categories[9]).sum()),('Complete_code_5-20', lambda x: (x == df3['Complete_code'].cat.categories[10]).sum()),('Complete_code_5-21', lambda x: (x == df3['Complete_code'].cat.categories[11]).sum()),('Complete_code_5-3', lambda x: (x == df3['Complete_code'].cat.categories[12]).sum()),('Complete_code_5-31', lambda x: (x == df3['Complete_code'].cat.categories[13]).sum()),('Complete_code_5-32', lambda x: (x == df3['Complete_code'].cat.categories[14]).sum()),('Complete_code_5-33', lambda x: (x == df3['Complete_code'].cat.categories[15]).sum()),('Complete_code_5-34', lambda x: (x == df3['Complete_code'].cat.categories[16]).sum()),('Complete_code_5-35', lambda x: (x == df3['Complete_code'].cat.categories[17]).sum()),('Complete_code_5-36', lambda x: (x == df3['Complete_code'].cat.categories[18]).sum()),('Complete_code_5-5', lambda x: (x == df3['Complete_code'].cat.categories[19]).sum()),('Complete_code_5-7', lambda x: (x == df3['Complete_code'].cat.categories[20]).sum()),('Complete_code_5-9', lambda x: (x == df3['Complete_code'].cat.categories[21]).sum()),('Complete_code_5-90', lambda x: (x == df3['Complete_code'].cat.categories[22]).sum()),('Complete_code_5-98', lambda x: (x == df3['Complete_code'].cat.categories[23]).sum()),('Complete_code_5-99', lambda x: (x == df3['Complete_code'].cat.categories[24]).sum()),
    ],
    
    'Master': [
        ('Master_mode', calculate_mode),
        ('Master_711-083', lambda x: (x == df3['Master'].cat.categories[0]).sum()),('Master_Master Assente', lambda x: (x == df3['Master'].cat.categories[1]).sum()),('Master_711-084', lambda x: (x == df3['Master'].cat.categories[2]).sum()), 
    ],
    
    'Speed': [ ('Speed_mean', 'mean') ],
    
    'Criticality': [
        ('Criticality_mode', calculate_mode),
        ('Criticality_ratio', lambda x: (x == df['Criticality'].cat.categories[1]).sum() / len(x) if len(x) > 0 else 0),
        ('Criticality_0', lambda x: (x == df3['Criticality'].cat.categories[0]).sum()),('Criticality_1', lambda x: (x == df3['Criticality'].cat.categories[1]).sum()),
    ],    
    
    'Colour': [
        ('Colour_mode', calculate_mode),
        ('Colour_azzurro', lambda x: (x == df3['Colour'].cat.categories[0]).sum()),('Colour_giallo', lambda x: (x == df3['Colour'].cat.categories[1]).sum()),
    ] 
}

In [84]:
# 4) set "Time" variable as index of the dataframe:
df3.set_index('Time', inplace=True)

In [85]:
# 5) Daily dataset:
daily_df_pdm = df3.resample('D').agg(agg_funcs)
daily_df_pdm.columns = [col[1] for col in daily_df_pdm.columns]
print(len(daily_df_pdm))

1050


In [86]:
# 6) Weekly dataset:
weekly_df_pdm = df3.resample('W').agg(agg_funcs)
weekly_df_pdm.columns = [col[1] for col in weekly_df_pdm.columns]
print(len(weekly_df_pdm))

151


In [87]:
# 7) reset the index:
daily_df_pdm.reset_index(inplace=True)
weekly_df_pdm.reset_index(inplace=True)

In [88]:
# 8) drop na rows --> .resample() list all days(/weeks) and if there are no obs for that day(/week) it fills the raw with 0 or NaN:
daily_def_pdm = daily_df_pdm.dropna(how='all', subset=['Source_mode'], inplace=False)
daily_def_pdm.reset_index(inplace=True)
daily_def_pdm = daily_def_pdm.drop(columns=['index'])
weekly_def_pdm = weekly_df_pdm.dropna(how='all', subset=['Source_mode'], inplace=False)
weekly_def_pdm.reset_index(inplace=True)
weekly_def = weekly_def_pdm.drop(columns=['index'])
print(f"n_na_daily is {len(daily_df_pdm) - len(daily_def_pdm)}")
print(f"n_na_weekly is {len(weekly_df_pdm) - len(weekly_def_pdm)}")

n_na_daily is 167
n_na_weekly is 6


In [89]:
# 9) check difference in days between rows for daily one (time series):
Diff_days = []
for i in range(len(daily_def_pdm)-1):
    diff = (daily_def_pdm.loc[i+1, 'Time']) - (daily_def_pdm.loc[i, 'Time'])
    Diff_days.append(diff)

print("Delta days:")
diff_days_counts = Counter(Diff_days)
ordered_diff_days_counts = sorted(diff_days_counts.items())
for diff_time, count in ordered_diff_days_counts[:10]:
    print(f"{diff_time}: {count} occurrences")

Delta days:
1 days 00:00:00: 812 occurrences
2 days 00:00:00: 43 occurrences
3 days 00:00:00: 14 occurrences
4 days 00:00:00: 5 occurrences
5 days 00:00:00: 2 occurrences
7 days 00:00:00: 2 occurrences
8 days 00:00:00: 1 occurrences
13 days 00:00:00: 1 occurrences
18 days 00:00:00: 1 occurrences
26 days 00:00:00: 1 occurrences


In [90]:
daily_def_pdm

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Master_Master Assente,Master_711-084,Speed_mean,Criticality_mode,Criticality_ratio,Criticality_0,Criticality_1,Colour_mode,Colour_azzurro,Colour_giallo
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,57,74,40.189781,0,0.430657,78,59,azzurro,74,63
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,84,40,15.732877,0,0.178082,120,26,azzurro,119,27
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,56,44,28.310000,0,0.190000,81,19,azzurro,79,21
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,133,109,25.550607,0,0.311741,170,77,azzurro,168,79
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,209,78,18.951389,0,0.256944,214,74,azzurro,212,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,20,6,25.826087,0,0.195652,37,9,azzurro,37,9
879,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,68,15,27.861789,0,0.154472,104,19,azzurro,104,19
880,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,36,7,21.573171,0,0.170732,68,14,azzurro,68,14
881,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,64,12,25.973913,0,0.113043,102,13,azzurro,99,16


In [91]:
weekly_def_pdm

,index,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,...,Master_Master Assente,Master_711-084,Speed_mean,Criticality_mode,Criticality_ratio,Criticality_0,Criticality_1,Colour_mode,Colour_azzurro,Colour_giallo
0,0,2020-11-15,711-084,139,491,711-084,5,148,51,151,...,330,267,26.896825,0,0.287302,449,181,azzurro,440,190
1,1,2020-11-22,711-084,357,664,711-084,5,166,117,232,...,601,279,20.416259,0,0.256611,759,262,azzurro,750,271
2,2,2020-11-29,711-084,287,604,711-084,49,97,31,204,...,520,240,22.481481,0,0.232323,684,207,azzurro,671,220
3,3,2020-12-06,711-084,225,604,711-084,9,82,41,239,...,462,294,29.559710,0,0.188179,673,156,azzurro,660,169
4,4,2020-12-13,711-084,102,444,711-084,4,84,25,129,...,278,229,19.360806,0,0.260073,404,142,azzurro,389,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,146,2023-09-03,711-083,266,92,711-083,27,33,77,0,...,228,38,24.047486,0,0.243017,271,87,azzurro,271,87
141,147,2023-09-10,711-083,315,167,711-083,79,55,56,23,...,255,70,21.919087,0,0.182573,394,88,azzurro,391,91
142,148,2023-09-17,711-083,150,114,711-083,29,20,33,42,...,145,66,22.223485,0,0.265152,194,70,azzurro,194,70
143,149,2023-09-24,711-083,374,219,711-083,51,65,60,56,...,327,102,27.760540,0,0.202361,473,120,azzurro,470,123


#### PDO dataset:

In [92]:
# 1) function to compute the mode:
def calculate_mode(series):
    try:
        return mode(series)
    except StatisticsError:
        return series.mode().iloc[0] if not series.mode().empty else None

In [93]:
# 2) function to simplify the typing procedure:
def labels_cat_var(df, column_name):
    result = ""
    for i, cat in enumerate(df[column_name].cat.categories):
        result += f"('{column_name}_{cat}', lambda x: (x == df['{column_name}'].cat.categories[{i}]).sum()),"
    return result

# and apply:
for col in df.columns:
    if isinstance(df[col].dtype, pd.CategoricalDtype):
        print(labels_cat_var(df,col), "\n", "\n")

('Source_711-083', lambda x: (x == df['Source'].cat.categories[0]).sum()),('Source_711-084', lambda x: (x == df['Source'].cat.categories[1]).sum()), 
 

('Name_710-131', lambda x: (x == df['Name'].cat.categories[0]).sum()),('Name_710-132', lambda x: (x == df['Name'].cat.categories[1]).sum()),('Name_710-135', lambda x: (x == df['Name'].cat.categories[2]).sum()),('Name_710-170', lambda x: (x == df['Name'].cat.categories[3]).sum()),('Name_711-083', lambda x: (x == df['Name'].cat.categories[4]).sum()),('Name_711-084', lambda x: (x == df['Name'].cat.categories[5]).sum()),('Name_710-000', lambda x: (x == df['Name'].cat.categories[6]).sum()), 
 

('Event_type_OFF', lambda x: (x == df['Event_type'].cat.categories[0]).sum()),('Event_type_ON', lambda x: (x == df['Event_type'].cat.categories[1]).sum()), 
 

('Type_PDO', lambda x: (x == df['Type'].cat.categories[0]).sum()),('Type_TRAIN_DRIVER', lambda x: (x == df['Type'].cat.categories[1]).sum()),('Type_WARNING', lambda x: (x == df['Type'].cat.cat

In [94]:
# 3) definition of aggregation functions to use:
agg_funcs = {
    'Source': [
        ('Source_mode', calculate_mode),
        ('Source_711-083', lambda x: (x == df['Source'].cat.categories[0]).sum()),('Source_711-084', lambda x: (x == df['Source'].cat.categories[1]).sum()),
    ],
    
    'Name': [
        ('Name_mode', calculate_mode),
        ('Name_710-131', lambda x: (x == df['Name'].cat.categories[0]).sum()),('Name_710-132', lambda x: (x == df['Name'].cat.categories[1]).sum()),('Name_710-135', lambda x: (x == df['Name'].cat.categories[2]).sum()),('Name_710-170', lambda x: (x == df['Name'].cat.categories[3]).sum()),('Name_711-083', lambda x: (x == df['Name'].cat.categories[4]).sum()),('Name_711-084', lambda x: (x == df['Name'].cat.categories[5]).sum()),
    ],
    
    'Type': [
        ('Type_mode', calculate_mode),
        ('Type_PDO', lambda x: (x == df['Type'].cat.categories[0]).sum()),('Type_TRAIN_DRIVER', lambda x: (x == df['Type'].cat.categories[1]).sum()),('Type_WARNING', lambda x: (x == df['Type'].cat.categories[2]).sum()), 
    ],

    'Complete_code': [
        ('Complete_code_mode', calculate_mode),
        ('Complete_code_2-19-1', lambda x: (x == df['Complete_code'].cat.categories[0]).sum()),('Complete_code_2-19-2', lambda x: (x == df['Complete_code'].cat.categories[1]).sum()),('Complete_code_5-1-1', lambda x: (x == df['Complete_code'].cat.categories[2]).sum()),('Complete_code_5-1-2', lambda x: (x == df['Complete_code'].cat.categories[3]).sum()),('Complete_code_5-10-1', lambda x: (x == df['Complete_code'].cat.categories[4]).sum()),('Complete_code_5-10-2', lambda x: (x == df['Complete_code'].cat.categories[5]).sum()),('Complete_code_5-11-1', lambda x: (x == df['Complete_code'].cat.categories[6]).sum()),('Complete_code_5-11-10', lambda x: (x == df['Complete_code'].cat.categories[7]).sum()),('Complete_code_5-11-11', lambda x: (x == df['Complete_code'].cat.categories[8]).sum()),('Complete_code_5-11-2', lambda x: (x == df['Complete_code'].cat.categories[9]).sum()),('Complete_code_5-11-3', lambda x: (x == df['Complete_code'].cat.categories[10]).sum()),('Complete_code_5-11-4', lambda x: (x == df['Complete_code'].cat.categories[11]).sum()),('Complete_code_5-11-5', lambda x: (x == df['Complete_code'].cat.categories[12]).sum()),('Complete_code_5-11-6', lambda x: (x == df['Complete_code'].cat.categories[13]).sum()),('Complete_code_5-11-7', lambda x: (x == df['Complete_code'].cat.categories[14]).sum()),('Complete_code_5-11-8', lambda x: (x == df['Complete_code'].cat.categories[15]).sum()),('Complete_code_5-11-9', lambda x: (x == df['Complete_code'].cat.categories[16]).sum()),('Complete_code_5-12-1', lambda x: (x == df['Complete_code'].cat.categories[17]).sum()),('Complete_code_5-13-1', lambda x: (x == df['Complete_code'].cat.categories[18]).sum()),('Complete_code_5-13-2', lambda x: (x == df['Complete_code'].cat.categories[19]).sum()),('Complete_code_5-13-3', lambda x: (x == df['Complete_code'].cat.categories[20]).sum()),('Complete_code_5-13-4', lambda x: (x == df['Complete_code'].cat.categories[21]).sum()),('Complete_code_5-14-1', lambda x: (x == df['Complete_code'].cat.categories[22]).sum()),('Complete_code_5-15-1', lambda x: (x == df['Complete_code'].cat.categories[23]).sum()),('Complete_code_5-16-1', lambda x: (x == df['Complete_code'].cat.categories[24]).sum()),('Complete_code_5-16-2', lambda x: (x == df['Complete_code'].cat.categories[25]).sum()),('Complete_code_5-17', lambda x: (x == df['Complete_code'].cat.categories[26]).sum()),('Complete_code_5-3-1', lambda x: (x == df['Complete_code'].cat.categories[27]).sum()),('Complete_code_5-3-10', lambda x: (x == df['Complete_code'].cat.categories[28]).sum()),('Complete_code_5-3-11', lambda x: (x == df['Complete_code'].cat.categories[29]).sum()),('Complete_code_5-3-12', lambda x: (x == df['Complete_code'].cat.categories[30]).sum()),('Complete_code_5-3-13', lambda x: (x == df['Complete_code'].cat.categories[31]).sum()),('Complete_code_5-3-14', lambda x: (x == df['Complete_code'].cat.categories[32]).sum()),('Complete_code_5-3-15', lambda x: (x == df['Complete_code'].cat.categories[33]).sum()),('Complete_code_5-3-16', lambda x: (x == df['Complete_code'].cat.categories[34]).sum()),('Complete_code_5-3-17', lambda x: (x == df['Complete_code'].cat.categories[35]).sum()),('Complete_code_5-3-18', lambda x: (x == df['Complete_code'].cat.categories[36]).sum()),('Complete_code_5-3-19', lambda x: (x == df['Complete_code'].cat.categories[37]).sum()),('Complete_code_5-3-2', lambda x: (x == df['Complete_code'].cat.categories[38]).sum()),('Complete_code_5-3-20', lambda x: (x == df['Complete_code'].cat.categories[39]).sum()),('Complete_code_5-3-21', lambda x: (x == df['Complete_code'].cat.categories[40]).sum()),('Complete_code_5-3-22', lambda x: (x == df['Complete_code'].cat.categories[41]).sum()),('Complete_code_5-3-23', lambda x: (x == df['Complete_code'].cat.categories[42]).sum()),('Complete_code_5-3-24', lambda x: (x == df['Complete_code'].cat.categories[43]).sum()),('Complete_code_5-3-25', lambda x: (x == df['Complete_code'].cat.categories[44]).sum()),('Complete_code_5-3-26', lambda x: (x == df['Complete_code'].cat.categories[45]).sum()),('Complete_code_5-3-27', lambda x: (x == df['Complete_code'].cat.categories[46]).sum()),('Complete_code_5-3-28', lambda x: (x == df['Complete_code'].cat.categories[47]).sum()),('Complete_code_5-3-29', lambda x: (x == df['Complete_code'].cat.categories[48]).sum()),('Complete_code_5-3-3', lambda x: (x == df['Complete_code'].cat.categories[49]).sum()),('Complete_code_5-3-30', lambda x: (x == df['Complete_code'].cat.categories[50]).sum()),('Complete_code_5-3-31', lambda x: (x == df['Complete_code'].cat.categories[51]).sum()),('Complete_code_5-3-32', lambda x: (x == df['Complete_code'].cat.categories[52]).sum()),('Complete_code_5-3-4', lambda x: (x == df['Complete_code'].cat.categories[53]).sum()),('Complete_code_5-3-5', lambda x: (x == df['Complete_code'].cat.categories[54]).sum()),('Complete_code_5-3-6', lambda x: (x == df['Complete_code'].cat.categories[55]).sum()),('Complete_code_5-3-7', lambda x: (x == df['Complete_code'].cat.categories[56]).sum()),('Complete_code_5-3-8', lambda x: (x == df['Complete_code'].cat.categories[57]).sum()),('Complete_code_5-3-9', lambda x: (x == df['Complete_code'].cat.categories[58]).sum()),('Complete_code_5-3-99', lambda x: (x == df['Complete_code'].cat.categories[59]).sum()),('Complete_code_5-31-1', lambda x: (x == df['Complete_code'].cat.categories[60]).sum()),('Complete_code_5-31-10', lambda x: (x == df['Complete_code'].cat.categories[61]).sum()),('Complete_code_5-31-11', lambda x: (x == df['Complete_code'].cat.categories[62]).sum()),('Complete_code_5-31-12', lambda x: (x == df['Complete_code'].cat.categories[63]).sum()),('Complete_code_5-31-13', lambda x: (x == df['Complete_code'].cat.categories[64]).sum()),('Complete_code_5-31-14', lambda x: (x == df['Complete_code'].cat.categories[65]).sum()),('Complete_code_5-31-15', lambda x: (x == df['Complete_code'].cat.categories[66]).sum()),('Complete_code_5-31-16', lambda x: (x == df['Complete_code'].cat.categories[67]).sum()),('Complete_code_5-31-2', lambda x: (x == df['Complete_code'].cat.categories[68]).sum()),('Complete_code_5-31-3', lambda x: (x == df['Complete_code'].cat.categories[69]).sum()),('Complete_code_5-31-4', lambda x: (x == df['Complete_code'].cat.categories[70]).sum()),('Complete_code_5-31-5', lambda x: (x == df['Complete_code'].cat.categories[71]).sum()),('Complete_code_5-31-6', lambda x: (x == df['Complete_code'].cat.categories[72]).sum()),('Complete_code_5-31-7', lambda x: (x == df['Complete_code'].cat.categories[73]).sum()),('Complete_code_5-31-8', lambda x: (x == df['Complete_code'].cat.categories[74]).sum()),('Complete_code_5-31-9', lambda x: (x == df['Complete_code'].cat.categories[75]).sum()),('Complete_code_5-32-1', lambda x: (x == df['Complete_code'].cat.categories[76]).sum()),('Complete_code_5-32-10', lambda x: (x == df['Complete_code'].cat.categories[77]).sum()),('Complete_code_5-32-11', lambda x: (x == df['Complete_code'].cat.categories[78]).sum()),('Complete_code_5-32-12', lambda x: (x == df['Complete_code'].cat.categories[79]).sum()),('Complete_code_5-32-13', lambda x: (x == df['Complete_code'].cat.categories[80]).sum()),('Complete_code_5-32-14', lambda x: (x == df['Complete_code'].cat.categories[81]).sum()),('Complete_code_5-32-15', lambda x: (x == df['Complete_code'].cat.categories[82]).sum()),('Complete_code_5-32-16', lambda x: (x == df['Complete_code'].cat.categories[83]).sum()),('Complete_code_5-32-2', lambda x: (x == df['Complete_code'].cat.categories[84]).sum()),('Complete_code_5-32-3', lambda x: (x == df['Complete_code'].cat.categories[85]).sum()),('Complete_code_5-32-4', lambda x: (x == df['Complete_code'].cat.categories[86]).sum()),('Complete_code_5-32-5', lambda x: (x == df['Complete_code'].cat.categories[87]).sum()),('Complete_code_5-32-6', lambda x: (x == df['Complete_code'].cat.categories[88]).sum()),('Complete_code_5-32-7', lambda x: (x == df['Complete_code'].cat.categories[89]).sum()),('Complete_code_5-32-8', lambda x: (x == df['Complete_code'].cat.categories[90]).sum()),('Complete_code_5-32-9', lambda x: (x == df['Complete_code'].cat.categories[91]).sum()),('Complete_code_5-33-1', lambda x: (x == df['Complete_code'].cat.categories[92]).sum()),('Complete_code_5-33-10', lambda x: (x == df['Complete_code'].cat.categories[93]).sum()),('Complete_code_5-33-11', lambda x: (x == df['Complete_code'].cat.categories[94]).sum()),('Complete_code_5-33-12', lambda x: (x == df['Complete_code'].cat.categories[95]).sum()),('Complete_code_5-33-13', lambda x: (x == df['Complete_code'].cat.categories[96]).sum()),('Complete_code_5-33-14', lambda x: (x == df['Complete_code'].cat.categories[97]).sum()),('Complete_code_5-33-15', lambda x: (x == df['Complete_code'].cat.categories[98]).sum()),('Complete_code_5-33-16', lambda x: (x == df['Complete_code'].cat.categories[99]).sum()),('Complete_code_5-33-2', lambda x: (x == df['Complete_code'].cat.categories[100]).sum()),('Complete_code_5-33-3', lambda x: (x == df['Complete_code'].cat.categories[101]).sum()),('Complete_code_5-33-4', lambda x: (x == df['Complete_code'].cat.categories[102]).sum()),('Complete_code_5-33-5', lambda x: (x == df['Complete_code'].cat.categories[103]).sum()),('Complete_code_5-33-6', lambda x: (x == df['Complete_code'].cat.categories[104]).sum()),('Complete_code_5-33-7', lambda x: (x == df['Complete_code'].cat.categories[105]).sum()),('Complete_code_5-33-8', lambda x: (x == df['Complete_code'].cat.categories[106]).sum()),('Complete_code_5-33-9', lambda x: (x == df['Complete_code'].cat.categories[107]).sum()),('Complete_code_5-34-1', lambda x: (x == df['Complete_code'].cat.categories[108]).sum()),('Complete_code_5-34-10', lambda x: (x == df['Complete_code'].cat.categories[109]).sum()),('Complete_code_5-34-11', lambda x: (x == df['Complete_code'].cat.categories[110]).sum()),('Complete_code_5-34-12', lambda x: (x == df['Complete_code'].cat.categories[111]).sum()),('Complete_code_5-34-13', lambda x: (x == df['Complete_code'].cat.categories[112]).sum()),('Complete_code_5-34-14', lambda x: (x == df['Complete_code'].cat.categories[113]).sum()),('Complete_code_5-34-15', lambda x: (x == df['Complete_code'].cat.categories[114]).sum()),('Complete_code_5-34-16', lambda x: (x == df['Complete_code'].cat.categories[115]).sum()),('Complete_code_5-34-2', lambda x: (x == df['Complete_code'].cat.categories[116]).sum()),('Complete_code_5-34-3', lambda x: (x == df['Complete_code'].cat.categories[117]).sum()),('Complete_code_5-34-4', lambda x: (x == df['Complete_code'].cat.categories[118]).sum()),('Complete_code_5-34-5', lambda x: (x == df['Complete_code'].cat.categories[119]).sum()),('Complete_code_5-34-6', lambda x: (x == df['Complete_code'].cat.categories[120]).sum()),('Complete_code_5-34-7', lambda x: (x == df['Complete_code'].cat.categories[121]).sum()),('Complete_code_5-34-8', lambda x: (x == df['Complete_code'].cat.categories[122]).sum()),('Complete_code_5-34-9', lambda x: (x == df['Complete_code'].cat.categories[123]).sum()),('Complete_code_5-34-99', lambda x: (x == df['Complete_code'].cat.categories[124]).sum()),('Complete_code_5-35-1', lambda x: (x == df['Complete_code'].cat.categories[125]).sum()),('Complete_code_5-35-10', lambda x: (x == df['Complete_code'].cat.categories[126]).sum()),('Complete_code_5-35-11', lambda x: (x == df['Complete_code'].cat.categories[127]).sum()),('Complete_code_5-35-12', lambda x: (x == df['Complete_code'].cat.categories[128]).sum()),('Complete_code_5-35-13', lambda x: (x == df['Complete_code'].cat.categories[129]).sum()),('Complete_code_5-35-14', lambda x: (x == df['Complete_code'].cat.categories[130]).sum()),('Complete_code_5-35-15', lambda x: (x == df['Complete_code'].cat.categories[131]).sum()),('Complete_code_5-35-16', lambda x: (x == df['Complete_code'].cat.categories[132]).sum()),('Complete_code_5-35-2', lambda x: (x == df['Complete_code'].cat.categories[133]).sum()),('Complete_code_5-35-3', lambda x: (x == df['Complete_code'].cat.categories[134]).sum()),('Complete_code_5-35-4', lambda x: (x == df['Complete_code'].cat.categories[135]).sum()),('Complete_code_5-35-5', lambda x: (x == df['Complete_code'].cat.categories[136]).sum()),('Complete_code_5-35-6', lambda x: (x == df['Complete_code'].cat.categories[137]).sum()),('Complete_code_5-35-7', lambda x: (x == df['Complete_code'].cat.categories[138]).sum()),('Complete_code_5-35-8', lambda x: (x == df['Complete_code'].cat.categories[139]).sum()),('Complete_code_5-35-9', lambda x: (x == df['Complete_code'].cat.categories[140]).sum()),('Complete_code_5-35-99', lambda x: (x == df['Complete_code'].cat.categories[141]).sum()),('Complete_code_5-36-1', lambda x: (x == df['Complete_code'].cat.categories[142]).sum()),('Complete_code_5-36-10', lambda x: (x == df['Complete_code'].cat.categories[143]).sum()),('Complete_code_5-36-11', lambda x: (x == df['Complete_code'].cat.categories[144]).sum()),('Complete_code_5-36-12', lambda x: (x == df['Complete_code'].cat.categories[145]).sum()),('Complete_code_5-36-13', lambda x: (x == df['Complete_code'].cat.categories[146]).sum()),('Complete_code_5-36-14', lambda x: (x == df['Complete_code'].cat.categories[147]).sum()),('Complete_code_5-36-15', lambda x: (x == df['Complete_code'].cat.categories[148]).sum()),('Complete_code_5-36-16', lambda x: (x == df['Complete_code'].cat.categories[149]).sum()),('Complete_code_5-36-2', lambda x: (x == df['Complete_code'].cat.categories[150]).sum()),('Complete_code_5-36-4', lambda x: (x == df['Complete_code'].cat.categories[151]).sum()),('Complete_code_5-36-5', lambda x: (x == df['Complete_code'].cat.categories[152]).sum()),('Complete_code_5-36-6', lambda x: (x == df['Complete_code'].cat.categories[153]).sum()),('Complete_code_5-36-7', lambda x: (x == df['Complete_code'].cat.categories[154]).sum()),('Complete_code_5-36-8', lambda x: (x == df['Complete_code'].cat.categories[155]).sum()),('Complete_code_5-36-9', lambda x: (x == df['Complete_code'].cat.categories[156]).sum()),('Complete_code_5-36-99', lambda x: (x == df['Complete_code'].cat.categories[157]).sum()),('Complete_code_5-5-1', lambda x: (x == df['Complete_code'].cat.categories[158]).sum()),('Complete_code_5-5-2', lambda x: (x == df['Complete_code'].cat.categories[159]).sum()),('Complete_code_5-5-3', lambda x: (x == df['Complete_code'].cat.categories[160]).sum()),('Complete_code_5-5-4', lambda x: (x == df['Complete_code'].cat.categories[161]).sum()),('Complete_code_5-5-5', lambda x: (x == df['Complete_code'].cat.categories[162]).sum()),('Complete_code_5-5-99', lambda x: (x == df['Complete_code'].cat.categories[163]).sum()),('Complete_code_5-7-1', lambda x: (x == df['Complete_code'].cat.categories[164]).sum()),('Complete_code_5-7-2', lambda x: (x == df['Complete_code'].cat.categories[165]).sum()),('Complete_code_5-7-3', lambda x: (x == df['Complete_code'].cat.categories[166]).sum()),('Complete_code_5-7-99', lambda x: (x == df['Complete_code'].cat.categories[167]).sum()),('Complete_code_5-9-1', lambda x: (x == df['Complete_code'].cat.categories[168]).sum()),('Complete_code_5-9-2', lambda x: (x == df['Complete_code'].cat.categories[169]).sum()),('Complete_code_5-9-3', lambda x: (x == df['Complete_code'].cat.categories[170]).sum()),('Complete_code_5-9-4', lambda x: (x == df['Complete_code'].cat.categories[171]).sum()),('Complete_code_5-9-5', lambda x: (x == df['Complete_code'].cat.categories[172]).sum()),('Complete_code_5-9-6', lambda x: (x == df['Complete_code'].cat.categories[173]).sum()),('Complete_code_5-9-99', lambda x: (x == df['Complete_code'].cat.categories[174]).sum()),('Complete_code_5-90-1', lambda x: (x == df['Complete_code'].cat.categories[175]).sum()),('Complete_code_5-90-2', lambda x: (x == df['Complete_code'].cat.categories[176]).sum()),('Complete_code_5-90-5', lambda x: (x == df['Complete_code'].cat.categories[177]).sum()),('Complete_code_5-90-6', lambda x: (x == df['Complete_code'].cat.categories[178]).sum()),('Complete_code_5-90-7', lambda x: (x == df['Complete_code'].cat.categories[179]).sum()),('Complete_code_5-98-1', lambda x: (x == df['Complete_code'].cat.categories[180]).sum()),('Complete_code_5-98-10', lambda x: (x == df['Complete_code'].cat.categories[181]).sum()),('Complete_code_5-98-11', lambda x: (x == df['Complete_code'].cat.categories[182]).sum()),('Complete_code_5-98-12', lambda x: (x == df['Complete_code'].cat.categories[183]).sum()),('Complete_code_5-98-13', lambda x: (x == df['Complete_code'].cat.categories[184]).sum()),('Complete_code_5-98-14', lambda x: (x == df['Complete_code'].cat.categories[185]).sum()),('Complete_code_5-98-15', lambda x: (x == df['Complete_code'].cat.categories[186]).sum()),('Complete_code_5-98-16', lambda x: (x == df['Complete_code'].cat.categories[187]).sum()),('Complete_code_5-98-17', lambda x: (x == df['Complete_code'].cat.categories[188]).sum()),('Complete_code_5-98-18', lambda x: (x == df['Complete_code'].cat.categories[189]).sum()),('Complete_code_5-98-19', lambda x: (x == df['Complete_code'].cat.categories[190]).sum()),('Complete_code_5-98-2', lambda x: (x == df['Complete_code'].cat.categories[191]).sum()),('Complete_code_5-98-20', lambda x: (x == df['Complete_code'].cat.categories[192]).sum()),('Complete_code_5-98-24', lambda x: (x == df['Complete_code'].cat.categories[193]).sum()),('Complete_code_5-98-25', lambda x: (x == df['Complete_code'].cat.categories[194]).sum()),('Complete_code_5-98-26', lambda x: (x == df['Complete_code'].cat.categories[195]).sum()),('Complete_code_5-98-27', lambda x: (x == df['Complete_code'].cat.categories[196]).sum()),('Complete_code_5-98-28', lambda x: (x == df['Complete_code'].cat.categories[197]).sum()),('Complete_code_5-98-29', lambda x: (x == df['Complete_code'].cat.categories[198]).sum()),('Complete_code_5-98-3', lambda x: (x == df['Complete_code'].cat.categories[199]).sum()),('Complete_code_5-98-30', lambda x: (x == df['Complete_code'].cat.categories[200]).sum()),('Complete_code_5-98-31', lambda x: (x == df['Complete_code'].cat.categories[201]).sum()),('Complete_code_5-98-32', lambda x: (x == df['Complete_code'].cat.categories[202]).sum()),('Complete_code_5-98-33', lambda x: (x == df['Complete_code'].cat.categories[203]).sum()),('Complete_code_5-98-7', lambda x: (x == df['Complete_code'].cat.categories[204]).sum()),('Complete_code_5-98-8', lambda x: (x == df['Complete_code'].cat.categories[205]).sum()),('Complete_code_5-98-9', lambda x: (x == df['Complete_code'].cat.categories[206]).sum()),('Complete_code_5-99-10', lambda x: (x == df['Complete_code'].cat.categories[207]).sum()),('Complete_code_5-99-11', lambda x: (x == df['Complete_code'].cat.categories[208]).sum()),('Complete_code_5-99-12', lambda x: (x == df['Complete_code'].cat.categories[209]).sum()),('Complete_code_5-99-13', lambda x: (x == df['Complete_code'].cat.categories[210]).sum()),('Complete_code_5-99-14', lambda x: (x == df['Complete_code'].cat.categories[211]).sum()),('Complete_code_5-99-5', lambda x: (x == df['Complete_code'].cat.categories[212]).sum()),('Complete_code_5-99-6', lambda x: (x == df['Complete_code'].cat.categories[213]).sum()),('Complete_code_5-99-7', lambda x: (x == df['Complete_code'].cat.categories[214]).sum()),('Complete_code_5-99-8', lambda x: (x == df['Complete_code'].cat.categories[215]).sum()),('Complete_code_5-99-9', lambda x: (x == df['Complete_code'].cat.categories[216]).sum()),('Complete_code_5-36-3', lambda x: (x == df['Complete_code'].cat.categories[217]).sum()),('Complete_code_5-90-4', lambda x: (x == df['Complete_code'].cat.categories[218]).sum()),('Complete_code_5-90-3', lambda x: (x == df['Complete_code'].cat.categories[219]).sum()),
    ],
    
    'Master': [
        ('Master_mode', calculate_mode),
        ('Master_711-083', lambda x: (x == df['Master'].cat.categories[0]).sum()),('Master_711-084', lambda x: (x == df['Master'].cat.categories[1]).sum()),('Master_Master Assente', lambda x: (x == df['Master'].cat.categories[2]).sum()), 
    ],
    
    'Speed': [ ('Speed_mean', 'mean') ],
    
    'Criticality': [
        ('Criticality_mode', calculate_mode),
        ('Criticality_ratio', lambda x: (x == df['Criticality'].cat.categories[1]).sum() / len(x) if len(x) > 0 else 0),
        ('Criticality_0', lambda x: (x == df['Criticality'].cat.categories[0]).sum()),('Criticality_1', lambda x: (x == df['Criticality'].cat.categories[1]).sum()),
    ],    
    
    'Colour': [
        ('Colour_mode', calculate_mode),
        ('Colour_azzurro', lambda x: (x == df['Colour'].cat.categories[0]).sum()),('Colour_giallo', lambda x: (x == df['Colour'].cat.categories[1]).sum()),
    ] 
}

In [95]:
# 4) set "Time" variable as index of the dataframe:
df.set_index('Time', inplace=True)

In [96]:
# 5) Daily dataset:
daily_df = df.resample('D').agg(agg_funcs)
daily_df.columns = [col[1] for col in daily_df.columns]
print(len(daily_df))

1050


In [97]:
# 6) Weekly dataset:
weekly_df = df.resample('W').agg(agg_funcs)
weekly_df.columns = [col[1] for col in weekly_df.columns]
print(len(weekly_df))

151


In [98]:
# 7) reset the index:
daily_df.reset_index(inplace=True)
weekly_df.reset_index(inplace=True)

In [99]:
# 8) drop na rows --> .resample() list all days(/weeks) and if there are no obs for that day(/week) it fills the raw with 0 or NaN:
daily_def = daily_df.dropna(how='all', subset=['Source_mode'], inplace=False)
daily_def.reset_index(inplace=True)
daily_def = daily_def.drop(columns=['index'])
weekly_def = weekly_df.dropna(how='all', subset=['Source_mode'], inplace=False)
weekly_def.reset_index(inplace=True)
weekly_def = weekly_def.drop(columns=['index'])
print(f"n_na_daily is {len(daily_df)-len(daily_def)}")
print(f"n_na_weekly is {len(weekly_df)-len(weekly_def)}")

n_na_daily is 167
n_na_weekly is 6


In [100]:
# 9) check difference in days between rows for daily one (time series):
Diff_days = []
for i in range(len(daily_def)-1):
    diff = (daily_def.loc[i+1, 'Time']) - (daily_def.loc[i, 'Time'])
    Diff_days.append(diff)

print("Delta days:")
diff_days_counts = Counter(Diff_days)
ordered_diff_days_counts = sorted(diff_days_counts.items())
for diff_time, count in ordered_diff_days_counts[:10]:
    print(f"{diff_time}: {count} occurrences")

Delta days:
1 days 00:00:00: 812 occurrences
2 days 00:00:00: 43 occurrences
3 days 00:00:00: 14 occurrences
4 days 00:00:00: 5 occurrences
5 days 00:00:00: 2 occurrences
7 days 00:00:00: 2 occurrences
8 days 00:00:00: 1 occurrences
13 days 00:00:00: 1 occurrences
18 days 00:00:00: 1 occurrences
26 days 00:00:00: 1 occurrences


In [101]:
daily_def

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Master_711-084,Master_Master Assente,Speed_mean,Criticality_mode,Criticality_ratio,Criticality_0,Criticality_1,Colour_mode,Colour_azzurro,Colour_giallo
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,57,74,40.189781,0,0.036496,132,5,azzurro,74,63
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,84,40,15.732877,0,0.000000,146,0,azzurro,119,27
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,56,44,28.310000,0,0.000000,100,0,azzurro,79,21
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,133,109,25.550607,0,0.174089,204,43,azzurro,168,79
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,209,78,18.951389,0,0.097222,260,28,azzurro,212,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,20,6,25.826087,0,0.000000,46,0,azzurro,37,9
879,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,68,15,27.861789,0,0.000000,123,0,azzurro,104,19
880,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,36,7,21.573171,0,0.024390,80,2,azzurro,68,14
881,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,64,12,25.973913,0,0.000000,115,0,azzurro,99,16


In [102]:
weekly_def

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Master_711-084,Master_Master Assente,Speed_mean,Criticality_mode,Criticality_ratio,Criticality_0,Criticality_1,Colour_mode,Colour_azzurro,Colour_giallo
0,2020-11-15,711-084,139,491,711-084,5,148,51,151,83,...,330,267,26.896825,0,0.076190,582,48,azzurro,440,190
1,2020-11-22,711-084,357,664,711-084,5,166,117,232,235,...,601,279,20.416259,0,0.056807,963,58,azzurro,750,271
2,2020-11-29,711-084,287,604,711-084,49,97,31,204,207,...,520,240,22.481481,0,0.015713,877,14,azzurro,671,220
3,2020-12-06,711-084,225,604,711-084,9,82,41,239,175,...,462,294,29.559710,0,0.033776,801,28,azzurro,660,169
4,2020-12-13,711-084,102,444,711-084,4,84,25,129,73,...,278,229,19.360806,0,0.082418,501,45,azzurro,389,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,2023-09-03,711-083,266,92,711-083,27,33,77,0,162,...,228,38,24.047486,0,0.000000,358,0,azzurro,271,87
141,2023-09-10,711-083,315,167,711-083,79,55,56,23,180,...,255,70,21.919087,0,0.020747,472,10,azzurro,391,91
142,2023-09-17,711-083,150,114,711-083,29,20,33,42,88,...,145,66,22.223485,0,0.045455,252,12,azzurro,194,70
143,2023-09-24,711-083,374,219,711-083,51,65,60,56,263,...,327,102,27.760540,0,0.015177,584,9,azzurro,470,123


### Daily dataset --> new variables:

#### - Season variable

In [103]:
# 1) def function:
def get_season(date):
  m = date.month
  x = m%12 // 3 + 1
  if x == 1:
    season = "Winter"
  if x == 2:
    season = "Spring"
  if x == 3:
    season = "Summer"
  if x == 4:
    season = "Autumn"
  return season

# 2) apply it:
#   a) PDO dataset:
daily_def['Season'] = daily_def['Time'].apply(lambda x: get_season(x))
daily_def['Season'] = daily_def['Season'].astype('category')
print(daily_def['Season'].value_counts())

#   b) PDM dataset:
daily_def_pdm['Season'] = daily_def_pdm['Time'].apply(lambda x: get_season(x))
daily_def_pdm['Season'] = daily_def_pdm['Season'].astype('category')
print(daily_def_pdm['Season'].value_counts())

Season
Spring    251
Summer    236
Autumn    200
Winter    196
Name: count, dtype: int64
Season
Spring    251
Summer    236
Autumn    200
Winter    196
Name: count, dtype: int64


In [104]:
daily_def

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Master_Master Assente,Speed_mean,Criticality_mode,Criticality_ratio,Criticality_0,Criticality_1,Colour_mode,Colour_azzurro,Colour_giallo,Season
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,74,40.189781,0,0.036496,132,5,azzurro,74,63,Autumn
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,40,15.732877,0,0.000000,146,0,azzurro,119,27,Autumn
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,44,28.310000,0,0.000000,100,0,azzurro,79,21,Autumn
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,109,25.550607,0,0.174089,204,43,azzurro,168,79,Autumn
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,78,18.951389,0,0.097222,260,28,azzurro,212,76,Autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,6,25.826087,0,0.000000,46,0,azzurro,37,9,Autumn
879,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,15,27.861789,0,0.000000,123,0,azzurro,104,19,Autumn
880,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,7,21.573171,0,0.024390,80,2,azzurro,68,14,Autumn
881,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,12,25.973913,0,0.000000,115,0,azzurro,99,16,Autumn


In [105]:
daily_def_pdm

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Master_711-084,Speed_mean,Criticality_mode,Criticality_ratio,Criticality_0,Criticality_1,Colour_mode,Colour_azzurro,Colour_giallo,Season
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,74,40.189781,0,0.430657,78,59,azzurro,74,63,Autumn
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,40,15.732877,0,0.178082,120,26,azzurro,119,27,Autumn
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,44,28.310000,0,0.190000,81,19,azzurro,79,21,Autumn
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,109,25.550607,0,0.311741,170,77,azzurro,168,79,Autumn
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,78,18.951389,0,0.256944,214,74,azzurro,212,76,Autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
878,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,6,25.826087,0,0.195652,37,9,azzurro,37,9,Autumn
879,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,15,27.861789,0,0.154472,104,19,azzurro,104,19,Autumn
880,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,7,21.573171,0,0.170732,68,14,azzurro,68,14,Autumn
881,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,12,25.973913,0,0.113043,102,13,azzurro,99,16,Autumn


#### - Number of services and KM done each day:

In [106]:
# 1) in order to obtain the count of services per day and also the km per day, a resample is applied on the routes dataset:
#print(routes.head())
routes_2 = routes.drop(columns=(['Data','Departure','Arrival','Train']))
print(routes_2.head())
print(routes_2.columns)

routes_2.set_index('Date', inplace=True)
print(routes_2.head())
print(routes_2.columns)

agg_funcs_2 = {
    'KM': [
        ('Services', 'size'),
        ('Total_KM', 'sum')
    ]}

daily_routes = routes_2.resample('D').agg(agg_funcs_2)
daily_routes.columns = [col[1] for col in daily_routes.columns]
print(len(daily_routes))
print(daily_routes)

daily_routes.reset_index(inplace=True)

daily_routes['Services'] = daily_routes['Services'].replace(0, np.nan)
print(daily_routes)

daily_routes_def = daily_routes.dropna(how='all', subset=['Services'], inplace=False)
daily_routes_def.reset_index(inplace=True)
daily_routes_def = daily_routes_def.drop(columns=['index'])
daily_routes_def['Services'] = daily_routes_def['Services'].astype(int)
print(daily_routes_def.info())
print(daily_routes_def)
print("\n")
print(f"n_na_daily is {len(daily_routes)-len(daily_routes_def)}")

Diff_days = []
for i in range(len(daily_routes_def)-1):
    diff = (daily_routes_def.loc[i+1, 'Date']) - (daily_routes_def.loc[i, 'Date'])
    Diff_days.append(diff)
print("\n")
print("Delta days:")
diff_days_counts = Counter(Diff_days)
ordered_diff_days_counts = sorted(diff_days_counts.items())
for diff_time, count in ordered_diff_days_counts[:10]:
    print(f"{diff_time}: {count} occurrences")

          KM       Date
index                  
0       79.2 2020-08-27
1       91.0 2020-08-27
2      851.2 2020-08-28
3        3.0 2020-08-28
4       37.6 2020-09-12
Index(['KM', 'Date'], dtype='object')
               KM
Date             
2020-08-27   79.2
2020-08-27   91.0
2020-08-28  851.2
2020-08-28    3.0
2020-09-12   37.6
Index(['KM'], dtype='object')
1127
            Services  Total_KM
Date                          
2020-08-27         2     170.2
2020-08-28         2     854.2
2020-08-29         0       0.0
2020-08-30         0       0.0
2020-08-31         0       0.0
...              ...       ...
2023-09-23        13     428.9
2023-09-24        10     491.0
2023-09-25        10     423.4
2023-09-26         8     385.0
2023-09-27         3     149.7

[1127 rows x 2 columns]
           Date  Services  Total_KM
0    2020-08-27       2.0     170.2
1    2020-08-28       2.0     854.2
2    2020-08-29       NaN       0.0
3    2020-08-30       NaN       0.0
4    2020-08-31       NaN

In [107]:
# 2) check the daily log data and daily routes data
print(len(daily_def))
print(len(daily_routes_def))
print("\n")
print(daily_def['Time'].head())
print("\n")
print(daily_routes_def['Date'].head())
print("\n")
print(daily_def['Time'].tail())
print("\n")
print(daily_routes_def['Date'].tail())

883
824


0   2020-11-12
1   2020-11-13
2   2020-11-14
3   2020-11-15
4   2020-11-16
Name: Time, dtype: datetime64[ns]


0   2020-08-27
1   2020-08-28
2   2020-09-12
3   2020-09-13
4   2020-09-14
Name: Date, dtype: datetime64[ns]


878   2023-09-23
879   2023-09-24
880   2023-09-25
881   2023-09-26
882   2023-09-27
Name: Time, dtype: datetime64[ns]


819   2023-09-23
820   2023-09-24
821   2023-09-25
822   2023-09-26
823   2023-09-27
Name: Date, dtype: datetime64[ns]


In [108]:
# 3) merge the 2 in order to add to daily log data the 2 new variables:
#   a) PDO dataset:
daily_def_2 = pd.merge(daily_def, daily_routes_def, left_on='Time', right_on='Date')
daily_def_2 = daily_def_2.drop(columns=['Date'])

#   b) PDM dataset:
daily_def_pdm_2 = pd.merge(daily_def_pdm, daily_routes_def, left_on='Time', right_on='Date')
daily_def_pdm_2 = daily_def_pdm_2.drop(columns=['Date'])

In [109]:
daily_def_2

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Criticality_mode,Criticality_ratio,Criticality_0,Criticality_1,Colour_mode,Colour_azzurro,Colour_giallo,Season,Services,Total_KM
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,0,0.036496,132,5,azzurro,74,63,Autumn,17,324.3
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,0,0.000000,146,0,azzurro,119,27,Autumn,18,338.4
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,0,0.000000,100,0,azzurro,79,21,Autumn,17,404.6
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,0,0.174089,204,43,azzurro,168,79,Autumn,16,402.8
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,0,0.097222,260,28,azzurro,212,76,Autumn,17,381.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,0,0.000000,46,0,azzurro,37,9,Autumn,13,428.9
763,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,0,0.000000,123,0,azzurro,104,19,Autumn,10,491.0
764,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,0,0.024390,80,2,azzurro,68,14,Autumn,10,423.4
765,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,0,0.000000,115,0,azzurro,99,16,Autumn,8,385.0


In [110]:
daily_def_pdm_2

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Criticality_mode,Criticality_ratio,Criticality_0,Criticality_1,Colour_mode,Colour_azzurro,Colour_giallo,Season,Services,Total_KM
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,0,0.430657,78,59,azzurro,74,63,Autumn,17,324.3
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,0,0.178082,120,26,azzurro,119,27,Autumn,18,338.4
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,0,0.190000,81,19,azzurro,79,21,Autumn,17,404.6
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,0,0.311741,170,77,azzurro,168,79,Autumn,16,402.8
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,0,0.256944,214,74,azzurro,212,76,Autumn,17,381.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,0,0.195652,37,9,azzurro,37,9,Autumn,13,428.9
763,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,0,0.154472,104,19,azzurro,104,19,Autumn,10,491.0
764,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,0,0.170732,68,14,azzurro,68,14,Autumn,10,423.4
765,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,0,0.113043,102,13,azzurro,99,16,Autumn,8,385.0


#### - Number of scheduled maintenance done up to each day

In [111]:
# 1) check dataset:
print(sched_maint.head())
# 'End_date' column is used
sched_maint_2 = sched_maint.drop(columns=(['Train', 'Level', 'Km', 'Start_date', 'Duration']))
print(sched_maint_2.info())
print(sched_maint_2)

               Train Type Level      Km Start_date   End_date Duration
0  TSR 040 (6 casse)  VIS     2  955258        NaT 2020-08-26   0 days
1  TSR 040 (6 casse)   VI     1  955258        NaT 2020-08-28   0 days
2  TSR 040 (6 casse)   R4     1  955258        NaT 2020-08-28   0 days
3  TSR 040 (6 casse)   RP     1  955258        NaT 2020-08-28   0 days
4  TSR 040 (6 casse)   VI     1  970756        NaT 2020-10-22   0 days
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Type      29 non-null     category      
 1   End_date  29 non-null     datetime64[ns]
dtypes: category(1), datetime64[ns](1)
memory usage: 437.0 bytes
None
   Type   End_date
0   VIS 2020-08-26
1    VI 2020-08-28
2    R4 2020-08-28
3    RP 2020-08-28
4    VI 2020-10-22
5    RT 2020-12-17
6    VI 2021-02-19
7    RT 2021-04-09
8    VI 2021-05-29
9    RT 2021-07-24
10   VI 2

In [112]:
# 2) create a dataframe with the same column of Date as the daily log:
sched_maint_appoggio = pd.DataFrame()
sched_maint_appoggio['Date'] = daily_def_2['Time']
print(sched_maint_appoggio)

          Date
0   2020-11-12
1   2020-11-13
2   2020-11-14
3   2020-11-15
4   2020-11-16
..         ...
762 2023-09-23
763 2023-09-24
764 2023-09-25
765 2023-09-26
766 2023-09-27

[767 rows x 1 columns]


In [113]:
# 3a) progressive count of each type of scheduled maintenance:
print(sched_maint_2['Type'].cat.categories)

count_R4, count_RD, count_RP, count_RT, count_VI, count_VIS = 0, 0, 0, 0, 0, 0
R4, RD, RP, RT, VI, VIS = [], [], [], [], [], []

Index(['R4', 'RD', 'RP', 'RT', 'VI', 'VIS'], dtype='object')


In [114]:
for i in range(len(sched_maint_appoggio)):      # prima itero sul daily log perchè devo capire quante manut fatte fino a ogni data QUI
    for j in range(len(sched_maint_2)):         # poi itero sulle manutenzioni per definire il count
        if i == 0:
            if sched_maint_2.loc[j, 'End_date'] <= sched_maint_appoggio.loc[i, 'Date']:
                if sched_maint_2.loc[j, 'Type'] == 'R4':
                    count_R4 += 1
                elif sched_maint_2.loc[j, 'Type'] == 'RD':
                    count_RD += 1
                elif sched_maint_2.loc[j, 'Type'] == 'RP':
                    count_RP += 1
                elif sched_maint_2.loc[j, 'Type'] == 'RT':
                    count_RT += 1
                elif sched_maint_2.loc[j, 'Type'] == 'VI':
                    count_VI += 1
                elif sched_maint_2.loc[j, 'Type'] == 'VIS':
                    count_VIS += 1
                    
        else:
            if (sched_maint_2.loc[j, 'End_date'] <= sched_maint_appoggio.loc[i, 'Date']) and (sched_maint_2.loc[j, 'End_date'] > sched_maint_appoggio.loc[i-1, 'Date']):
                if sched_maint_2.loc[j, 'Type'] == 'R4':
                    count_R4 += 1
                elif sched_maint_2.loc[j, 'Type'] == 'RD':
                    count_RD += 1
                elif sched_maint_2.loc[j, 'Type'] == 'RP':
                    count_RP += 1
                elif sched_maint_2.loc[j, 'Type'] == 'RT':
                    count_RT += 1
                elif sched_maint_2.loc[j, 'Type'] == 'VI':
                    count_VI += 1
                elif sched_maint_2.loc[j, 'Type'] == 'VIS':
                    count_VIS += 1
    R4.append(count_R4)
    RD.append(count_RD)
    RP.append(count_RP)
    RT.append(count_RT)
    VI.append(count_VI)
    VIS.append(count_VIS)

In [115]:
print(R4)
print(len(R4))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [116]:
print(RD)
print(len(RD))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [117]:
print(RP)
print(len(RP))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [118]:
print(RT)
print(len(RT))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [119]:
print(VI)
print(len(VI))

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 

In [120]:
print(VIS)
print(len(VIS))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [121]:
# 3b) progressive count all scheduled maintenances:
count_sched_maint = 0
Corr_sched = []

for i in range(len(sched_maint_appoggio)):
    for j in range(len(sched_maint_2)):
        if i == 0:
            if sched_maint_2.loc[j, 'End_date'] <= sched_maint_appoggio.loc[i, 'Date']:
                count_sched_maint += 1                 
        else:
            if (sched_maint_2.loc[j, 'End_date'] <= sched_maint_appoggio.loc[i, 'Date']) and (sched_maint_2.loc[j, 'End_date'] > sched_maint_appoggio.loc[i-1, 'Date']):
                count_sched_maint += 1
    Corr_sched.append(count_sched_maint)

In [122]:
# 4) add to the df appoggio:
sched_maint_appoggio['R4_progr_count'] = R4
sched_maint_appoggio['RD_progr_count'] = RD
sched_maint_appoggio['RP_progr_count'] = RP
sched_maint_appoggio['RT_progr_count'] = RT
sched_maint_appoggio['VI_progr_count'] = VI
sched_maint_appoggio['VIS_progr_count'] = VIS
sched_maint_appoggio['Sched_maint_progr'] = Corr_sched

print(sched_maint_appoggio.head())

        Date  R4_progr_count  RD_progr_count  RP_progr_count  RT_progr_count  \
0 2020-11-12               1               0               1               0   
1 2020-11-13               1               0               1               0   
2 2020-11-14               1               0               1               0   
3 2020-11-15               1               0               1               0   
4 2020-11-16               1               0               1               0   

   VI_progr_count  VIS_progr_count  Sched_maint_progr  
0               2                1                  5  
1               2                1                  5  
2               2                1                  5  
3               2                1                  5  
4               2                1                  5  


In [123]:
# 5) merge the 2:
#   a) PDO dataset:
daily_def_3 = pd.merge(daily_def_2, sched_maint_appoggio, left_on='Time', right_on='Date')
daily_def_3 = daily_def_3.drop(columns=['Date'])

#   b) PDM dataset:
daily_def_pdm_3 = pd.merge(daily_def_pdm_2, sched_maint_appoggio, left_on='Time', right_on='Date')
daily_def_pdm_3 = daily_def_pdm_3.drop(columns=['Date'])

In [124]:
daily_def_3

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Season,Services,Total_KM,R4_progr_count,RD_progr_count,RP_progr_count,RT_progr_count,VI_progr_count,VIS_progr_count,Sched_maint_progr
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,Autumn,17,324.3,1,0,1,0,2,1,5
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,Autumn,18,338.4,1,0,1,0,2,1,5
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,Autumn,17,404.6,1,0,1,0,2,1,5
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,Autumn,16,402.8,1,0,1,0,2,1,5
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,Autumn,17,381.1,1,0,1,0,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,Autumn,13,428.9,1,1,2,10,11,1,26
763,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,Autumn,10,491.0,1,1,2,10,11,1,26
764,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,Autumn,10,423.4,1,1,2,10,11,1,26
765,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,Autumn,8,385.0,1,1,2,10,11,1,26


In [125]:
daily_def_pdm_3

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Season,Services,Total_KM,R4_progr_count,RD_progr_count,RP_progr_count,RT_progr_count,VI_progr_count,VIS_progr_count,Sched_maint_progr
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,Autumn,17,324.3,1,0,1,0,2,1,5
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,Autumn,18,338.4,1,0,1,0,2,1,5
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,Autumn,17,404.6,1,0,1,0,2,1,5
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,Autumn,16,402.8,1,0,1,0,2,1,5
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,Autumn,17,381.1,1,0,1,0,2,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,Autumn,13,428.9,1,1,2,10,11,1,26
763,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,Autumn,10,491.0,1,1,2,10,11,1,26
764,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,Autumn,10,423.4,1,1,2,10,11,1,26
765,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,Autumn,8,385.0,1,1,2,10,11,1,26


#### - Number of corrective maintenance done up to each day

In [126]:
# 1) check dataset:
print(corr_maint.head())
# 'End_date' column is used
corr_maint_2 = corr_maint['End_date']
print(corr_maint_2.info())
print(corr_maint_2)

         Avviso   Train    Wagon    N. invent. Type  Treno Group_maint_source  \
0  100001697429  R040-6  710-170  948367101706   T2    222           Trazione   
1  100001700911  R040-6  710-170  948367101706   T2  20228           Trazione   
2  100001701703  R040-6  710-131  948367101318   T2   8206           Trazione   
3  100001701841  R040-6  710-132  948367101326   T2   2730           Trazione   
4  100001703623  R040-6  710-170  948367101706   T2    813           Trazione   

                               Descrizione  \
0                    05.15 AutoEscl TCU 00   
1  Az. escluso per guasto controllo az. 00   
2                    05.15 AutoEscl TCU 00   
3          Inverter di trazione Escluso 00   
4                    05.15 AutoEscl TCU 00   

                          Testo esteso               Testo interv.  \
0                                 None  Ripristino livello glicole   
1  * autoesclusione azionamento da tcu            rabbocco glicole   
2                         

In [127]:
# 2) create a dataframe with the same column of Date as the daily log:
corr_maint_appoggio = pd.DataFrame()
corr_maint_appoggio['Date'] = daily_def_2['Time']
print(corr_maint_appoggio)

          Date
0   2020-11-12
1   2020-11-13
2   2020-11-14
3   2020-11-15
4   2020-11-16
..         ...
762 2023-09-23
763 2023-09-24
764 2023-09-25
765 2023-09-26
766 2023-09-27

[767 rows x 1 columns]


In [128]:
# 3) progressive count of each type of corrective maintenance:
count_corr_maint = 0
Corr_maint = []

for i in range(len(corr_maint_appoggio)):
    for j in range(len(corr_maint_2)):
        if i == 0:
            if corr_maint_2.loc[j] <= corr_maint_appoggio.loc[i, 'Date']:
                count_corr_maint += 1                 
        else:
            if (corr_maint_2.loc[j] <= corr_maint_appoggio.loc[i, 'Date']) and (corr_maint_2.loc[j] > corr_maint_appoggio.loc[i-1, 'Date']):
                count_corr_maint += 1
    Corr_maint.append(count_corr_maint)

In [129]:
print(Corr_maint)
print(len(Corr_maint))

[6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 13, 13, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15

In [130]:
# 4) add to the df appoggio:
corr_maint_appoggio['Corr_maint_progr'] = Corr_maint
print(corr_maint_appoggio.head())

# 5) merge the 2:
#   a) PDO dataset:
daily_def_4 = pd.merge(daily_def_3, corr_maint_appoggio, left_on='Time', right_on='Date')
daily_def_4 = daily_def_4.drop(columns=['Date'])

#   b) PDM dataset:
daily_def_pdm_4 = pd.merge(daily_def_pdm_3, corr_maint_appoggio, left_on='Time', right_on='Date')
daily_def_pdm_4 = daily_def_pdm_4.drop(columns=['Date'])

        Date  Corr_maint_progr
0 2020-11-12                 6
1 2020-11-13                 6
2 2020-11-14                 6
3 2020-11-15                 6
4 2020-11-16                 6


In [131]:
daily_def_4

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Services,Total_KM,R4_progr_count,RD_progr_count,RP_progr_count,RT_progr_count,VI_progr_count,VIS_progr_count,Sched_maint_progr,Corr_maint_progr
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,17,324.3,1,0,1,0,2,1,5,6
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,18,338.4,1,0,1,0,2,1,5,6
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,17,404.6,1,0,1,0,2,1,5,6
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,16,402.8,1,0,1,0,2,1,5,6
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,17,381.1,1,0,1,0,2,1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,13,428.9,1,1,2,10,11,1,26,25
763,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,10,491.0,1,1,2,10,11,1,26,25
764,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,10,423.4,1,1,2,10,11,1,26,25
765,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,8,385.0,1,1,2,10,11,1,26,25


In [132]:
daily_def_pdm_4

,Time,Source_mode,Source_711-083,Source_711-084,Name_mode,Name_710-131,Name_710-132,Name_710-135,Name_710-170,Name_711-083,...,Services,Total_KM,R4_progr_count,RD_progr_count,RP_progr_count,RT_progr_count,VI_progr_count,VIS_progr_count,Sched_maint_progr,Corr_maint_progr
0,2020-11-12,711-084,36,101,710-132,0,42,21,31,15,...,17,324.3,1,0,1,0,2,1,5,6
1,2020-11-13,711-084,69,77,711-083,0,19,19,20,50,...,18,338.4,1,0,1,0,2,1,5,6
2,2020-11-14,711-084,22,78,711-084,3,19,9,25,10,...,17,404.6,1,0,1,0,2,1,5,6
3,2020-11-15,711-084,12,235,711-084,2,68,2,75,8,...,16,402.8,1,0,1,0,2,1,5,6
4,2020-11-16,711-084,22,266,710-132,5,97,9,76,8,...,17,381.1,1,0,1,0,2,1,5,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
762,2023-09-23,711-083,31,15,711-083,6,2,2,8,23,...,13,428.9,1,1,2,10,11,1,26,25
763,2023-09-24,711-083,76,47,711-083,4,0,6,8,66,...,10,491.0,1,1,2,10,11,1,26,25
764,2023-09-25,711-083,53,29,711-083,11,1,10,8,32,...,10,423.4,1,1,2,10,11,1,26,25
765,2023-09-26,711-083,80,35,711-083,5,0,8,16,67,...,8,385.0,1,1,2,10,11,1,26,25


#### Save the 2 datasets:

In [133]:
# save the dataset:
def copy_df(original_df):
    new_df = original_df.copy()
    new_df = new_df.astype(original_df.dtypes)
    return new_df

df040_final_3 = copy_df(daily_def_4)
df040_final_3.to_csv("Final_dataset//TSR_040_daily.csv", index=False)
df040_final_3.to_parquet('Final_dataset//TSR_040_daily.parquet', engine='fastparquet')

df040_final_3_pdm = copy_df(daily_def_pdm_4)
df040_final_3_pdm.to_csv("Final_dataset//TSR_040_daily_PDM.csv", index=False)
df040_final_3_pdm.to_parquet('Final_dataset//TSR_040_daily_PDM.parquet', engine='fastparquet')

## EDA

daily_def_4 = daily_def_4[(daily_def_4['Time'] > '2020-08-28') & (daily_def_4['Time'] < '2023/01/25')]
daily_def_4 = daily_def_4.reset_index(drop=True)
len(daily_def_4)

print(daily_def_4['Source_mode'].describe())
print(daily_def_4['Source_mode'].value_counts())
print("\n")

print(daily_def_4['Source_711-083'].describe())
print(daily_def_4['Source_711-084'].describe())
print("\n")

print(daily_def_4['Name_mode'].describe())
print(daily_def_4['Name_mode'].value_counts())
print("\n")

print(daily_def_4['Name_710-131'].describe())
print(daily_def_4['Name_710-132'].describe())
print(daily_def_4['Name_710-135'].describe())
print(daily_def_4['Name_710-170'].describe())
print(daily_def_4['Name_711-083'].describe())
print(daily_def_4['Name_711-084'].describe())
print("\n")

print(daily_def_4['Type_mode'].describe())
print(daily_def_4['Type_mode'].value_counts())
print("\n")

print(daily_def_4['Type_PDO'].describe())
print(daily_def_4['Type_TRAIN_DRIVER'].describe())
print(daily_def_4['Type_WARNING'].describe())
print("\n")

print(daily_def_4['Complete_code_mode'].describe())
print(daily_def_4['Complete_code_mode'].value_counts())
print("\n")

print(daily_def_4['Speed_mean'].describe())

print(daily_def_4['Criticality_mode'].describe())
print(daily_def_4['Criticality_mode'].value_counts())
print("\n")
print(daily_def_4['Criticality_ratio'].describe())
print("\n")
print(daily_def_4['Criticality_0'].describe())
print("\n")
print(daily_def_4['Criticality_1'].describe())
print("\n")
print(daily_def_4['Criticality_0'].sum())
print(daily_def_4['Criticality_1'].sum())

print(daily_def_4['Colour_mode'].describe())
print(daily_def_4['Colour_mode'].value_counts())
print("\n")
print(daily_def_4['Colour_azzurro'].describe())
print("\n")
print(daily_def_4['Colour_giallo'].describe())

print(daily_def_4['Season'].describe())
print(daily_def_4['Season'].value_counts())

print(daily_def_4['Services'].describe())

print(daily_def_4['Total_KM'].describe())

print(daily_def_4['R4_progr_count'].describe())
print("\n")
print(daily_def_4['RD_progr_count'].describe())
print("\n")
print(daily_def_4['RP_progr_count'].describe())
print("\n")
print(daily_def_4['RT_progr_count'].describe())
print("\n")
print(daily_def_4['VI_progr_count'].describe())
print("\n")
print(daily_def_4['VIS_progr_count'].describe())

print(daily_def_4['Corr_maint_progr'].describe())

# some plots:
#!pip install seaborn
import seaborn as sns
import matplotlib.pyplot as plt

sns.displot(data=daily_def_4, x="Type_PDO", binwidth=1)
sns.displot(data=daily_def_4, x="Type_PDO", kind="kde")
sns.displot(data=daily_def_4, x="Spead_mean", kind="kde")
sns.displot(data=daily_def_4, x="Services", kind="kde")
sns.displot(data=daily_def_4, x="Total_KM", kind="kde")
sns.displot(data=daily_def_4, x="Corr_maint_progr", binwidth=1)

In [134]:
# da scartare:

In [135]:
'''
for i in range(5):                                      # prima itero sul daily log perchè devo capire quante manut fatte fino a ogni data QUI
    for j in range(len(sched_maint_2)):         # poi itero sulle manutenzioni per definire il count
        if i == 0:
            print(i)
            print(f"daily log data: {sched_maint_appoggio.loc[i,'Date']}")
            print(f"maint data: {sched_maint_2.loc[j, 'End_date']}")
            if sched_maint_2.loc[j, 'End_date'] <= sched_maint_appoggio.loc[i, 'Date']:
                if sched_maint_2.loc[j, 'Type'] == 'R4':
                    count_R4 += 1
                    R4.append(count_R4)
                    print(f"R4: {R4}")
                elif sched_maint_2.loc[j, 'Type'] == 'RD':
                    count_RD += 1
                    RD.append(count_RD)
                    print(f"RD: {RD}")
                elif sched_maint_2.loc[j, 'Type'] == 'RP':
                    count_RP += 1
                    RP.append(count_RP)
                    print(f"RP: {RP}")
                elif sched_maint_2.loc[j, 'Type'] == 'RT':
                    count_RT += 1
                    RT.append(count_RT)
                    print(f"RT: {RT}")
                elif sched_maint_2.loc[j, 'Type'] == 'VI':
                    count_VI += 1
                    VI.append(count_VI)
                    print(f"VI: {VI}")
                elif sched_maint_2.loc[j, 'Type'] == 'VIS':
                    count_VIS += 1
                    VIS.append(count_VIS)
                    print(f"VIS: {VIS}")          
        else:
            print(i)
            print(f"daily log data: {sched_maint_appoggio.loc[i,'Date']}")
            print(f"maint data: {sched_maint_2.loc[j, 'End_date']}")
            if (sched_maint_2.loc[j, 'End_date'] <= sched_maint_appoggio.loc[i, 'Date']) and (sched_maint_2.loc[j, 'End_date'] > sched_maint_appoggio.loc[i-1, 'Date']):
                print("yes")
                if sched_maint_2.loc[j, 'Type'] == 'R4':
                    count_R4 += 1
                    R4.append(count_R4)
                    print(f"R4: {R4}")
                elif sched_maint_2.loc[j, 'Type'] == 'RD':
                    count_RD += 1
                    RD.append(count_RD)
                    print(f"RD: {RD}")
                elif sched_maint_2.loc[j, 'Type'] == 'RP':
                    count_RP += 1
                    RP.append(count_RP)
                    print(f"RP: {RP}")
                elif sched_maint_2.loc[j, 'Type'] == 'RT':
                    count_RT += 1
                    RT.append(count_RT)
                    print(f"RT: {RT}")
                elif sched_maint_2.loc[j, 'Type'] == 'VI':
                    count_VI += 1
                    VI.append(count_VI)
                    print(f"VI: {VI}")
                elif sched_maint_2.loc[j, 'Type'] == 'VIS':
                    count_VIS += 1
                    VIS.append(count_VIS)
                    print(f"VIS: {VIS}")
'''

'\nfor i in range(5):                                      # prima itero sul daily log perchè devo capire quante manut fatte fino a ogni data QUI\n    for j in range(len(sched_maint_2)):         # poi itero sulle manutenzioni per definire il count\n        if i == 0:\n            print(i)\n            print(f"daily log data: {sched_maint_appoggio.loc[i,\'Date\']}")\n            print(f"maint data: {sched_maint_2.loc[j, \'End_date\']}")\n            if sched_maint_2.loc[j, \'End_date\'] <= sched_maint_appoggio.loc[i, \'Date\']:\n                if sched_maint_2.loc[j, \'Type\'] == \'R4\':\n                    count_R4 += 1\n                    R4.append(count_R4)\n                    print(f"R4: {R4}")\n                elif sched_maint_2.loc[j, \'Type\'] == \'RD\':\n                    count_RD += 1\n                    RD.append(count_RD)\n                    print(f"RD: {RD}")\n                elif sched_maint_2.loc[j, \'Type\'] == \'RP\':\n                    count_RP += 1\n         

In [136]:
'''
for i in (1,2,3,4):                                    
    for j in range(len(sched_maint_2)):
        end_date = sched_maint_2.loc[j, 'End_date']
        print(end_date)
        current_date = sched_maint_appoggio.loc[i, 'Date']
        print(current_date)
        prev_date = sched_maint_appoggio.loc[i - 1, 'Date']
        print(prev_date)
        if end_date <= current_date and end_date > prev_date:
            print("yes")
        else:
            print("no")
'''

'\nfor i in (1,2,3,4):                                    \n    for j in range(len(sched_maint_2)):\n        end_date = sched_maint_2.loc[j, \'End_date\']\n        print(end_date)\n        current_date = sched_maint_appoggio.loc[i, \'Date\']\n        print(current_date)\n        prev_date = sched_maint_appoggio.loc[i - 1, \'Date\']\n        print(prev_date)\n        if end_date <= current_date and end_date > prev_date:\n            print("yes")\n        else:\n            print("no")\n'

In [137]:
'''
for i in (1,2,3,4):                                    
    for j in range(len(sched_maint_2['Type'])):
        if sched_maint_appoggio.loc[i-1, 'Date'] < sched_maint_2.loc[j, 'End_date'] <= sched_maint_appoggio.loc[i, 'Date']:
            print("yes")
        else:
            print("no")
'''

'\nfor i in (1,2,3,4):                                    \n    for j in range(len(sched_maint_2[\'Type\'])):\n        if sched_maint_appoggio.loc[i-1, \'Date\'] < sched_maint_2.loc[j, \'End_date\'] <= sched_maint_appoggio.loc[i, \'Date\']:\n            print("yes")\n        else:\n            print("no")\n'